# Correct the distorted images

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import json

def align_images_multiple_references(reference_image_paths, distorted_image_path):
    # Load the distorted image
    dist_img = cv2.imread(distorted_image_path)
    dist_gray = cv2.cvtColor(dist_img, cv2.COLOR_BGR2GRAY)

    # Initialize SIFT
    sift = cv2.SIFT_create()

    # Aggregate keypoints and descriptors from all reference images
    ref_keypoints = []
    ref_descriptors = []
    for ref_path in reference_image_paths:
        ref_img = cv2.imread(ref_path)
        ref_gray = cv2.cvtColor(ref_img, cv2.COLOR_BGR2GRAY)
        keypoints, descriptors = sift.detectAndCompute(ref_gray, None)
        ref_keypoints.extend(keypoints)
        if descriptors is not None:
            if len(ref_descriptors) == 0:
                ref_descriptors = descriptors
            else:
                ref_descriptors = np.vstack((ref_descriptors, descriptors))

    # Detect keypoints and descriptors in the distorted image
    keypoints_dist, descriptors_dist = sift.detectAndCompute(dist_gray, None)

    # Match features using BFMatcher
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    matches = bf.match(ref_descriptors, descriptors_dist)

    # Sort matches by distance
    matches = sorted(matches, key=lambda x: x.distance)

    # Extract matched keypoints
    ref_pts = np.float32([ref_keypoints[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dist_pts = np.float32([keypoints_dist[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    # Estimate homography
    matrix, mask = cv2.findHomography(dist_pts, ref_pts, cv2.RANSAC, 5.0)

    # Warp the distorted image
    h, w = dist_img.shape[:2]
    aligned_image = cv2.warpPerspective(dist_img, matrix, (w, h))

    # Plot the results
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.title("Distorted Image")
    plt.imshow(cv2.cvtColor(dist_img, cv2.COLOR_BGR2RGB))
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.title("Aligned Image")
    plt.imshow(cv2.cvtColor(aligned_image, cv2.COLOR_BGR2RGB))
    plt.axis("off")

    plt.tight_layout()
    plt.show()

    return matrix

def transform_image_with_homography(image_path, homography_matrix):
    # Load the image
    image = cv2.imread(image_path)
    h, w = image.shape[:2]  # Get the size from the image itself

    # Apply the homography transformation
    transformed_image = cv2.warpPerspective(image, homography_matrix, (w, h))

    return transformed_image

def reverse_transform_image_with_homography(transformed_image, homography_matrix):
    """
    Reverse the homography transformation on an image.

    Parameters:
        transformed_image: The image that was transformed.
        homography_matrix: The homography matrix used for the forward transformation.

    Returns:
        The reversed (original) image.
    """
    # Calculate the inverse of the homography matrix
    inverse_homography_matrix = np.linalg.inv(homography_matrix)
    
    # Get the size of the transformed image
    h, w = transformed_image.shape[:2]
    
    # Apply the inverse homography transformation
    original_image = cv2.warpPerspective(transformed_image, inverse_homography_matrix, (w, h))
    
    return original_image

def save_homography_matrix(matrix, file_path):
    # Convert the NumPy array to a list for JSON serialization
    matrix_list = matrix.tolist()
    with open(file_path, 'w') as f:
        json.dump(matrix_list, f)

def process_dataset_images(dataset_folder):
    images_folder = os.path.join(dataset_folder, "images")
    
    for image_name in os.listdir(images_folder):
        print(image_name)
        image_path = os.path.join(images_folder, image_name)

        reference_image_paths = [
            './Dataset 2/images/000003.png', './Dataset 2/images/000010.png', './Dataset 2/images/000016.png', './Dataset 2/images/000021.png', './Dataset 2/images/000022.png', './Dataset 2/images/000028.png', './Dataset 2/images/000030.png', './Dataset 2/images/000033.png', './Dataset 2/images/000046.png', './Dataset 2/images/000051.png', './Dataset 2/images/000053.png', './Dataset 2/images/000060.png', './Dataset 2/images/000061.png', './Dataset 2/images/000064.png',
            './Dataset 2/images/000072.png', './Dataset 2/images/000081.png', './Dataset 2/images/000096.png', 
        ]  # Add more references
        #distorted_image_path = './Dataset 2/images/000083.png'
        homography_matrix = align_images_multiple_references(reference_image_paths, image_path)

        # Transform the image
        transformed_image = transform_image_with_homography(image_path, homography_matrix)

        if not cv2.imwrite(f"./calibrated-images/{image_name}", transformed_image):
            print(f"Failed to save image: {image_name}")

        save_homography_matrix(homography_matrix, f"./calibrated-images/{image_name.replace('png', 'json')}")

        # Plot the original and transformed images
        #original_image = cv2.imread(image_path)
        #plt.figure(figsize=(12, 6))
        #plt.subplot(1, 2, 1)
        #plt.title("Original Image")
        #plt.imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
        #plt.axis("off")

        #plt.subplot(1, 2, 2)
        #plt.title("Transformed Image")
        #plt.imshow(cv2.cvtColor(transformed_image, cv2.COLOR_BGR2RGB))
        #plt.axis("off")

        #plt.tight_layout()
        #plt.show()

# Example usage
dataset_folder = './Dataset 2'  # Replace with the actual dataset folder
process_dataset_images(dataset_folder)

# Predict masks

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import os
import json

def load_homography_matrix(file_path):
    # Load the JSON file and convert the list back to a NumPy array
    with open(file_path, 'r') as f:
        matrix_list = json.load(f)
    return np.array(matrix_list)

def reverse_transform_image_with_homography(transformed_image, homography_matrix):
    """
    Reverse the homography transformation on an image.

    Parameters:
        transformed_image: The image that was transformed.
        homography_matrix: The homography matrix used for the forward transformation.

    Returns:
        The reversed (original) image.
    """
    # Calculate the inverse of the homography matrix
    inverse_homography_matrix = np.linalg.inv(homography_matrix)
    
    # Get the size of the transformed image
    h, w = transformed_image.shape[:2]
    
    # Apply the inverse homography transformation
    original_image = cv2.warpPerspective(transformed_image, inverse_homography_matrix, (w, h))
    
    return original_image

def color_thresholding(image, lower_bounds, upper_bounds):
    """
    Perform color thresholding to segment the globe.

    Parameters:
        image (numpy.ndarray): Input image in BGR format.
        lower_bounds (list of numpy.ndarray): List of lower HSV color bounds.
        upper_bounds (list of numpy.ndarray): List of upper HSV color bounds.

    Returns:
        numpy.ndarray: Binary mask highlighting the selected colors.
    """
    # Convert image to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = np.zeros(hsv_image.shape[:2], dtype=np.uint8)
    # Apply each color threshold and combine the masks
    for lower_bound, upper_bound in zip(lower_bounds, upper_bounds):
        mask |= cv2.inRange(hsv_image, lower_bound, upper_bound)
    return mask

def refined_circle_detection_with_mask(image, color_mask):
    """
    Detect a single circular shape in the image using a color mask to focus the detection.

    Parameters:
        image (numpy.ndarray): Input image in BGR format.
        color_mask (numpy.ndarray): Binary mask obtained from color thresholding.

    Returns:
        numpy.ndarray: Binary mask with the detected circle.
    """
    # Apply the mask to the image
    masked_image = cv2.bitwise_and(image, image, mask=color_mask)
    # Convert to grayscale
    gray_image = cv2.cvtColor(masked_image, cv2.COLOR_BGR2GRAY)
    # Apply Gaussian blur to reduce noise
    blurred_image = cv2.GaussianBlur(gray_image, (9, 9), 2)
    plt.figure(figsize=(6, 6))
    plt.imshow(blurred_image, cmap='gray')
    plt.title(f"Thresholding Result {i:06d}")
    plt.axis("off")
    plt.show()
    # Detect circles using HoughCircles with adjusted parameters
    #circles = cv2.HoughCircles(
    #    blurred_image,
    #    cv2.HOUGH_GRADIENT_ALT,
    #    dp=1.2,
    #    minDist=200,
    #    param1=100,
    #    param2=40,
    #    minRadius=200,
    #    maxRadius=500,
    #)
    
    circles = cv2.HoughCircles(
        blurred_image,
        cv2.HOUGH_GRADIENT_ALT,
        dp=2.3,          # Resolution of the accumulator (adjust if necessary)
        minDist=200,     # Minimum distance between detected circle centers
        param1=200,      # Higher threshold for Canny edge detection
        param2=0.30,      # Threshold for circle detection in accumulator
        minRadius=55,   # Minimum circle radius (adapt based on the image)
        maxRadius=444    # Maximum circle radius
    )
    # Create a binary mask with the same dimensions as the input image
    mask = np.zeros_like(gray_image)
    if circles is not None:
        # Get the first (most prominent) circle
        circle = np.round(circles[0, 0]).astype("int")
        x, y, r = circle
        # Draw the circle on the mask
        cv2.circle(mask, (x, y), r, 255, -1)
    return mask

def compute_roc_curve(predicted_mask, ground_truth_mask):
    """
    Compute ROC curve and AUC for a predicted mask and ground truth mask.

    Parameters:
        predicted_mask (numpy.ndarray): Predicted binary mask.
        ground_truth_mask (numpy.ndarray): Ground truth binary mask.

    Returns:
        tuple: (fpr, tpr, roc_auc) False Positive Rate, True Positive Rate, and Area Under Curve.
    """
    # Binarize the masks
    predicted_binary = (predicted_mask > 0).astype(np.uint8).flatten()
    ground_truth_binary = (ground_truth_mask > 0).astype(np.uint8).flatten()

    # Compute ROC curve
    fpr, tpr, _ = roc_curve(ground_truth_binary, predicted_binary)
    roc_auc = auc(fpr, tpr)

    return fpr, tpr, roc_auc

# Dataset paths
images_folder = './Dataset 2/images/'
corrected_images_folder = './calibrated-images/'
masks_folder = './Dataset 2/masks/'

# Bounds for color thresholding
lower_bounds = [
    np.array([100, 50, 50]),  # Lower bound of blue in HSV
    #np.array([0, 0, 200]),    # Lower bound of white in HSV
]
upper_bounds = [
    np.array([140, 255, 255]),  # Upper bound of blue in HSV
    #np.array([180, 50, 255]),   # Upper bound of white in HSV
]

# Lists to store ROC values
all_fpr = []
all_tpr = []
all_auc = []

# Loop through all images and masks
for i in range(100):
    image_path = os.path.join(images_folder, f"{i:06d}.png")
    corrected_image_path = os.path.join(corrected_images_folder, f"{i:06d}.png")
    mask_path = os.path.join(masks_folder, f"{i:06d}.png")

    # Read image and ground truth mask
    image = cv2.imread(image_path)
    corrected_image = cv2.imread(corrected_image_path)
    ground_truth_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    loaded_matrix = load_homography_matrix(corrected_image_path.replace("png", "json"))

    # Generate color mask
    color_mask = color_thresholding(corrected_image, lower_bounds, upper_bounds)

    # Plot the thresholding result
    plt.figure(figsize=(6, 6))
    plt.imshow(color_mask, cmap='gray')
    plt.title(f"Thresholding Result {i:06d}")
    plt.axis("off")
    plt.show()

    # Refine circle detection with color mask
    predicted_mask = reverse_transform_image_with_homography(refined_circle_detection_with_mask(image, color_mask), loaded_matrix)

    # Compute ROC curve
    fpr, tpr, roc_auc = compute_roc_curve(predicted_mask, ground_truth_mask)

    # Store the results
    all_fpr.append(fpr)
    all_tpr.append(tpr)
    all_auc.append(roc_auc)

    # Plot image, ground truth mask, and predicted mask
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 3, 1)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(f"Image {i:06d}")
    plt.axis("off")

    plt.subplot(1, 3, 2)
    plt.imshow(ground_truth_mask, cmap='gray')
    plt.title("Ground Truth Mask")
    plt.axis("off")

    plt.subplot(1, 3, 3)
    plt.imshow(predicted_mask, cmap='gray')
    plt.title("Predicted Mask")
    plt.axis("off")

    plt.tight_layout()
    plt.show()

# Compute average ROC curve (mean TPR for each FPR)
mean_fpr = np.linspace(0, 1, 100)
mean_tpr = np.zeros_like(mean_fpr)
for fpr, tpr in zip(all_fpr, all_tpr):
    mean_tpr += np.interp(mean_fpr, fpr, tpr)
mean_tpr /= len(all_tpr)
mean_auc = auc(mean_fpr, mean_tpr)

# Plot average ROC curve
plt.figure()
plt.plot(mean_fpr, mean_tpr, label=f"Mean ROC (AUC = {mean_auc:.2f})")
plt.title("Average ROC Curve for Test Set")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right")
plt.show()

# Print mean AUC
print(f"Mean AUC: {mean_auc:.2f}")


In [ ]:
# Python code to detect an arrow (seven-sided shape) from an image. 
import numpy as np 
import cv2 

# Reading image 
img2 = cv2.imread('./calibrated-images/000000.png', cv2.IMREAD_COLOR) 

# Reading same image in another variable and 
# converting to gray scale. 
img = cv2.imread('./calibrated-images/000000.png', cv2.IMREAD_GRAYSCALE) 

# Converting image to a binary image 
# (black and white only image). 
_,threshold = cv2.threshold(img, 110, 255, 
							cv2.THRESH_BINARY) 

# Detecting shapes in image by selecting region 
# with same colors or intensity. 
contours,_=cv2.findContours(threshold, cv2.RETR_TREE, 
							cv2.CHAIN_APPROX_SIMPLE) 

# Searching through every region selected to 
# find the required polygon. 
for cnt in contours : 
	area = cv2.contourArea(cnt) 

	# Shortlisting the regions based on there area. 
	if area > 400: 
		approx = cv2.approxPolyDP(cnt, 
								0.009 * cv2.arcLength(cnt, True), True) 

		# Checking if the no. of sides of the selected region is 7. 
		if(len(approx) == 7): 
			cv2.drawContours(img2, [approx], 0, (0, 0, 255), 5) 

# Showing the image along with outlined arrow. 
cv2.imshow('image2', img2) 

# Exiting the window if 'q' is pressed on the keyboard. 
if cv2.waitKey(0) & 0xFF == ord('q'): 
	cv2.destroyAllWindows() 
